In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

w.columns.name = "Factor"

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

h.columns.name = SETTING["sample_alias"]

In [ ]:
for w_or_h, dataframe in (("w", w.T), ("h", h)):

    k_hcc = ccal.hierarchical_consensus_cluster_with_ks(
        dataframe,
        SETTING["hcc_ks"],
        1,
        n_job=SETTING["n_job"],
        distance_function=ccal.compute_correlation_distance_between_2_1d_arrays,
        n_clustering=SETTING["hcc_n_clustering"],
        plot_dataframe=dataframe.shape[1] < SETTING["plot_cluster_max_size"],
        directory_path=PATH["{}|hcc/".format(w_or_h)],
    )

In [ ]:
w_or_h = "w"

new_clusters = ()

In [ ]:
if len(new_clusters):

    cluster_x_element = pd.read_csv(
        PATH["{}|cluster_x_element.tsv".format(w_or_h)], sep="\t", index_col=0
    )

    print(cluster_x_element.index)

    if (
        cluster_x_element.index.str.lstrip("C").astype(int).sort_values()
        == cluster_x_element.index.str.lstrip("C").astype(int)
    ).all():

        cluster_x_element.index = pd.Index(
            ("C{}".format(i) for i in new_clusters), name=cluster_x_element.index.name
        )

        ccal.clean_and_write_dataframe_to_tsv(
            cluster_x_element,
            cluster_x_element.index.name,
            PATH["{}|cluster_x_element.tsv".format(w_or_h)],
        )

        print(cluster_x_element.index)

    else:

        print("Indices may be already updated.")